# Demonstration of Mat Renderer

In [ ]:
# Path stores material svBRDF maps
maps_path = "assets/rusty_metal"
# Light configurations
light_position = (0.0, 0.0, 10.0)
light_color = (23.47, 21.31, 20.79)
light_power = 10.0

In [ ]:
from matrenderer.io import load_svbrdf_maps, show_maps
import matplotlib.pyplot as plt
from matrenderer.render import Renderer, Light
from matrenderer.helpers import create_single_batch_maps, create_rendered_maps
import pathlib

maps = load_svbrdf_maps(pathlib.Path(maps_path).absolute())
show_maps(maps)

In [ ]:
# Rendering

lights = [Light(light_position, light_color, light_power)]
r = Renderer(lights=lights, gamma=1.0)
batched_maps = create_single_batch_maps(maps)
color, ambient, radiance, diffuse, specular = r.render(batched_maps)

rendered_maps = create_rendered_maps(color[0], ambient[0], radiance[0]/radiance[0].max(), diffuse[0], specular[0])
show_maps(rendered_maps)

In [ ]:
# Larger rendered result
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
plt.imshow((color[0].permute(1,2,0)).cpu().numpy())